In [46]:
# imports
import warnings
warnings.filterwarnings("ignore")

import os

# transform
import numpy as np
import pandas as pd

# visualize 
import matplotlib.pyplot as plt
import seaborn as sns

# working with dates
from datetime import datetime

# modeling
import statsmodels.api as sm
from statsmodels.tsa.api import Holt, ExponentialSmoothing

# evaluate
from sklearn.metrics import mean_squared_error
from math import sqrt

# cross validation split
from sklearn.model_selection import StratifiedKFold

# Exercises

> ## **Using saas.csv or log data from API usage**

In [47]:
def get_saas_data():
    '''
    Checks for a local cache of tsa_store_data.csv and if not present will run the get_store_data() function which acquires data from Codeup's mysql server
    '''
    filename = 'saas.csv'
    if os.path.isfile(filename):
        df = pd.read_csv(filename)
    else:
        print(f'Search for it, I am not going to....')
    return df

In [48]:
df = get_saas_data()

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 790458 entries, 0 to 790457
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Month_Invoiced     790458 non-null  object 
 1   Customer_Id        790458 non-null  int64  
 2   Invoice_Id         790458 non-null  int64  
 3   Subscription_Type  790458 non-null  float64
 4   Amount             790458 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 30.2+ MB


In [50]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Customer_Id,790458.0,1.012913e+06,7591.991578,1000000.0,1006405.00,1012797.0,1019154.00,1029399.0
Invoice_Id,790458.0,5.395228e+06,228185.713880,5000000.0,5197614.25,5395228.5,5592842.75,5790457.0
Subscription_Type,790458.0,5.992349e-01,0.950380,0.0,0.00,0.0,1.00,4.0
Amount,790458.0,2.996174e+00,4.751901,0.0,0.00,0.0,5.00,20.0


In [51]:
df.head()

,Month_Invoiced,Customer_Id,Invoice_Id,Subscription_Type,Amount
0,2014-01-31,1000000,5000000,0.0,0.0
1,2014-01-31,1000001,5000001,2.0,10.0
2,2014-01-31,1000002,5000002,0.0,0.0
3,2014-01-31,1000003,5000003,0.0,0.0
4,2014-01-31,1000004,5000004,1.0,5.0


## Cleaning up column names

In [52]:
# cleaned up columns names prior to conversion
df.columns = [col.lower() for col in df.columns]
df.head()

,month_invoiced,customer_id,invoice_id,subscription_type,amount
0,2014-01-31,1000000,5000000,0.0,0.0
1,2014-01-31,1000001,5000001,2.0,10.0
2,2014-01-31,1000002,5000002,0.0,0.0
3,2014-01-31,1000003,5000003,0.0,0.0
4,2014-01-31,1000004,5000004,1.0,5.0


In [53]:
df.month_invoiced = pd.to_datetime(df.month_invoiced)

In [54]:
df = df.set_index('month_invoiced').sort_values(by='month_invoiced')
df.head

<bound method NDFrame.head of                 customer_id  invoice_id  subscription_type  amount
month_invoiced                                                    
2014-01-31          1000000     5000000                0.0     0.0
2014-01-31          1013336     5013336                0.0     0.0
2014-01-31          1013335     5013335                0.0     0.0
2014-01-31          1013334     5013334                0.0     0.0
2014-01-31          1013333     5013333                1.0     5.0
...                     ...         ...                ...     ...
2017-12-31          1012069     5781185                0.0     0.0
2017-12-31          1012072     5781186                0.0     0.0
2017-12-31          1012074     5781187                1.0     5.0
2017-12-31          1012047     5781176                3.0    15.0
2017-12-31          1029399     5790457                0.0     0.0

[790458 rows x 4 columns]>

In [55]:
# returning date range of data
print('Date Range:', df.index.min(), 'to', df.index.max())
print('Shape:', df.shape)

Date Range: 2014-01-31 00:00:00 to 2017-12-31 00:00:00
Shape: (790458, 4)


In [56]:
# Percentage based split

train_size = int(len(df) * .5)
validate_size = int(len(df) * .3)
test_size = int(len(df) - train_size - validate_size)
validate_end_index = train_size + validate_size

# split into train, validation, test
train = df[: train_size]
validate = df[train_size : validate_end_index]
test = df[validate_end_index : ]

In [41]:
# Human based split

train = df.loc['2014':'2015']
validate = df.loc['2016']
test = df.loc['2017']

train.shape, validate.shape, test.shape

((431894, 4), (186275, 4), (172289, 4))

In [42]:
# Cross Validation based

#-------starified random sampling----------------

# # 5 folds in cross validation; model will be trained on each fold and evaluated on the remaining
# # if data is biggger add more folds; may be computionally expensive
# kfold = StratifiedKFold(n_splits=5)

# # split data into train and validation folds.
# X_train, X_val, y_train, y_val = [], [], [], []
# for train.index ,val.index, in kfold.split(X, y):
#     X_train.append(X[train_index])
#     X_val.ppend(X[val_index])
#     y_train.append(y[train.index])
#     y_val.append(y[val.index])

# ------------------------------------------------

In [43]:
def wrangle_saas():
    df = get_saas_data()
    df.columns = [col.lower() for col in df.columns]
    df.month_invoiced = pd.to_datetime(df.month_invoiced)
    df = df.set_index('month_invoiced').sort_values(by='month_invoiced')
    train = df.loc['2014':'2015']
    val = df.loc['2016']
    test = df.loc['2017']

    return train, val, test

In [44]:
train, val, test = wrangle_saas()

train.shape, val.shape, test.shape

((431894, 4), (186275, 4), (172289, 4))

# 1. Split data (train/test) and resample by any period, except daily, and aggregate using the sum.

# 2. Forecast, plot and evaluate using each at least 4 of the methods we discussed:

##  Last Observed Value

##  Simple Average

##  Moving Average

## Holt's Linear Trend

## Holt's Seasonal Trend

## Based on previous year/month/etc., this is up to you.

# Bonus: 

### 1. Using the store item demand data, create a forecast of sales_total and quantity for 2018 using the Previous Cycle approach. .

### 2. Predict 2018 total monthly sales for a single store and/or item by creating a model using prophet. 3. Return a dataframe with the month, store_id, y-hat, and the confidence intervals (y-hat lower, y-hat upper). 4. Plot the 2018 monthly sales predictions.